## Æther Composition Priority
| z-index | Group | Item(s) |
| --- | --- | --- |
| 0 | Background | Background color |
| 1 | Background | Tilemap |
| 2 | Interaction | Non-player sprites |
| 3 | Interaction | Player sprite |
| 4 | Interaction | Projectiles |
| 5 | Effects | Visual effects |
| 6 | Overlay | UI |


## Steps:

### Layer 0: Background 
---
- **Goal: Print a solid color background to the terminal display.**
  - [x] Print a single color to the terminal.
  - [x] Fills the terminal by default.
  - [ ] ~~Never exceeds terminal bounds~~
  - [x] Can be resized smaller than the terminal
  - [x] Use 256-color extended ANSI codes.
  - [x] Keep a `term_utils.py` module for common terminal operations.
- **Classes Needed**
  - [x] NyxEntity
  - [x] NyxEntityManager
  - [x] NyxComponentStore
  - [x] NyxComponent (ABC)
  - [x] BackgroundColorComponent
  - [x] NyxSystem (ABC)
  - [x] RendererSystem
  - [x] AetherCompositor
  - [x] HemeraRenderer

- After a few trials, working up through the composition layers seems the best way to approach the problem. 
- So far so good through the `NyxSystem` class. Next is working on rendering collection.
- Render system collects and sends the prioritized entity dict to Aether
- Basic HemeraRenderer created, rendering a pixel to the screen

Next: Aether -> Hemera

### Layer 1: Tilemap 
---
- **Goal**
  - [x] Print an array of tiles to the terminal
  - [x] Fill zeros with bg color
  - [x] Cache/buffer tilemap after compute
  - [x] Store tilemaps in a resource or store by ID
- **Classes Needed**
  - [x] TilesetStore
  - [x] TilemapComponent
  - [x] Tilemap
  - [x] TilemapSystem
  - [x] SceneComponent
  - [x] PositionComponent

- Decided to move `Store`-type dictionaries to class attributes for easy access. `TilesetStore` has been made with class attributes.
- SceneComponent allows for a global container/flag
- PositionComponent should allow for scrolling

### Intermediate Goal: Subpixel Rendering
---
- **Goal**
  - [x] Update HemeraTermFx to use subpixel drawing
  - [x] Adjust the current calculations to match the doubled height resolution.
  - [x] Print 256-color tilemap in subpixels to the terminal. 

- Used simple list iterations for the actual printing, for now.
- Explore if its easier or faster to slice the ndarray before iterating.